# Scattering Phase Functions

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pymiecoated
import dsharp_opac as opacity

sns.set({'figure.dpi':200})

Define a plotting helper function

In [ ]:
def plot_logpolar(ax, theta, r_, bullseye=None, **kwargs):
    min10 = np.log10(np.min(r_))
    max10 = np.log10(np.max(r_))
    if bullseye is None:
        bullseye = min10 - np.log10(0.5 * np.min(r_))
    r = np.log10(r_) - min10 + bullseye
    ax.plot(theta, r, **kwargs)
    l = np.arange(np.floor(min10), max10)
    ax.set_rticks(l - min10 + bullseye) 
    ax.set_yticklabels(["$10^{{{:.0f}}}$".format(x) for x in l],fontdict={'fontsize':'xx-small'})
    ax.set_rlim(0, max10 - min10 + bullseye)
    return ax

Define the Henyey-Greenstein phase function

In [ ]:
def hg(mu, g):
    return (1 - g**2) / (1 + g**2 - 2 * g * mu)**1.5

## Calculate liquid water phase function

Set wavelength and sizes

In [ ]:
lam = np.array([0.55e-4])      # wavelength
a   = np.array([1e-4, 10e-4])  # particle radius

Calculate Mie opacities

In [ ]:
dc = opacity.diel_segelstein_water()
rho_s = dc.rho
result = opacity.get_opacities(a, lam, rho_s=rho_s, diel_const=dc, n_angle=500)

Make polar and regular plots of the phase function and overplot the HG phase function

In [ ]:
f1,ax1 = plt.subplots(1, 2, figsize=(12, 5), subplot_kw=dict(projection='polar'))
f2,ax2 = plt.subplots(1, 2, figsize=(12, 3))

for ia in range(len(a)):
    g     = result['g'][ia, 0]
    S1    = result['S1'][ia, 0, :]
    S2    = result['S2'][ia, 0, :]
    theta = result['theta']
    angle = theta * np.pi / 180.
    mu    = np.cos(angle)
    m     = 4 * np.pi / 3 * rho_s * a[ia]**3

    sig_s = result['q_sca'][ia, 0] * np.pi * a[ia]**2
    sig_a = result['q_abs'][ia, 0] * np.pi * a[ia]**2

    S11   = 0.5 * (S1 * S1.conjugate() + S2 * S2.conjugate()).real
    P11   = lam**2 / (np.pi * sig_s) * S11


    plot_logpolar(ax1[ia], angle, hg(mu,g), label='Henyey-Greenstein')
    plot_logpolar(ax1[ia], angle, P11, label='Mie calculation',lw=1)


    ax2[ia].semilogy(theta, hg(mu,g), label='Henyey-Greenstein')
    ax2[ia].semilogy(theta, P11, label='Mie calculation',lw=1)


for _ax in ax1:
    _ax.set_thetamax(180)
    _ax.set_xticks(np.linspace(0,np.pi,5))
    _ax.legend();
    
for _ax in ax2:
    _ax.set_xticks([0, 50, 100, 150])
    _ax.legend();

## Calculate Müller Matrix

In [ ]:
theta = result['theta']
k_sca = result['k_sca']
S1    = result['S1']
S2    = result['S2']
lam   = result['lam']
g     = result['g']

Calculate the masses and the Müller matrix

In [ ]:
m  = 4 * np.pi / 3 * rho_s * a**3
MM = opacity.calculate_mueller_matrix(lam, m, S1, S2, theta=theta, k_sca=k_sca)
zscat = MM['zscat']

In [ ]:
mu = np.cos(np.pi * theta / 180)

For some selected sizes and wavelengths: check conventions of $Z_{11}$.

In [ ]:
ia = 1
ilam = 0

kk = 2 * np.pi * np.trapz(zscat[ia, ilam, ::-1, 0], x=mu[::-1])
gg = 2 * np.pi / kk * np.trapz(mu[::-1] * zscat[ia, ilam, ::-1, 0], x=mu[::-1])


print('k_sca[Z11] = {:.6g}'.format(ks))
print('k_sca[Mie] = {:.6g}'.format(k_sca[ia, ilam]))
print('g[Z11]     = {:.6g}'.format(gg))
print('g[Mie]     = {:.6g}'.format(g[ia, ilam]))